# Visualización de redes

## Evolución temporal de la red de contratación (Áncash 2004-2024)
- Bipartita: Gobierno Regional (1 nodo) - Empresas (RUC)

In [3]:
from pathlib import Path
import pandas as pd, numpy as np, networkx as nx, json, re

# ----- Parámetros -----
RUTA_ARCHIVO = "contratos_ancash_all.xlsx"   # en esta carpeta
HOJA_EXCEL = 0
REGION_NOMBRE = "ANCASH"                     # se normaliza a mayúsculas sin tildes
ANIO_MIN, ANIO_MAX = 2004, 2024
PRORRATEAR_MONTO_CONSORCIO = False
SALIDA = Path("docs")   

In [4]:
# ============ Utilidades de limpieza ============
def _norm_region(s):
    if pd.isna(s): return s
    s2 = str(s).strip().upper()
    for a,b in zip("ÁÉÍÓÚ","AEIOU"): s2 = s2.replace(a,b)
    return s2

def _norm_emp(x):
    if pd.isna(x): return np.nan
    s = str(x).strip()
    s = s.replace("RUC","").replace(":","").replace(" ","")
    return s if s else np.nan

def _clean_monto(x):
    if pd.isna(x): return 0.0
    s = str(x).strip().replace("S/","").replace("s/","").replace("soles","").replace("SOLES","").strip()
    # 437.479.13 -> 437479.13
    if s.count(".") > 1:
        parts = s.split("."); s = "".join(parts[:-1]) + "." + parts[-1]
    # 17800,50 -> 17800.50
    if "," in s and "." not in s:
        s = s.replace(",", ".")
    # 1,234,567.89 -> 1234567.89
    if s.count(",") > 0 and s.count(".") == 1:
        s = s.replace(",", "")
    s = s.replace(" ", "")
    try:
        return float(s)
    except:
        t = "".join(re.findall(r"[0-9.,]", s))
        if t.count(".") > 1:
            parts = t.split("."); t = "".join(parts[:-1]) + "." + parts[-1]
        t = t.replace(",", "")
        try: return float(t)
        except: return 0.0

In [5]:
# ============ Carga ============
df = pd.read_excel(RUTA_ARCHIVO, sheet_name=HOJA_EXCEL)
df["FECHA"] = pd.to_datetime(df["FECHA"], dayfirst=True, errors="coerce")
df["ANIO"]  = df["FECHA"].dt.year
df["REGION_NORM"] = df["REGION"].apply(_norm_region)
REGION_OBJ = _norm_region(REGION_NOMBRE)

emp_cols = [c for c in df.columns if str(c).upper().startswith("EMPRESA")]
if not emp_cols:
    raise ValueError("No se encontraron columnas EMPRESA_n en el archivo.")

for c in emp_cols: df[c] = df[c].apply(_norm_emp)
df["MONTO"] = df["MONTO"].apply(_clean_monto)

df = df[(df["REGION_NORM"]==REGION_OBJ) & (df["ANIO"].between(ANIO_MIN, ANIO_MAX))].reset_index(drop=True)
if df.empty:
    raise ValueError("No hay registros para la región/años especificados.")

df["CONTRATO_ID"] = df.index.astype(str) + "_" + df["REGION_NORM"] + "_" + df["FECHA"].astype(str)
df["N_EMP_CONTRATO"] = df[emp_cols].notna().sum(axis=1).astype(int).replace(0,1)

# Formato largo: una fila por empresa vinculada a cada contrato
rows = []
for _, r in df.iterrows():
    anio = int(r["ANIO"]); region = r["REGION_NORM"]; monto = float(r["MONTO"])
    nemp = int(r["N_EMP_CONTRATO"]); cid = r["CONTRATO_ID"]
    for c in emp_cols:
        e = r[c]
        if pd.notna(e) and str(e).strip():
            m = (monto/nemp) if PRORRATEAR_MONTO_CONSORCIO else monto
            rows.append((anio, region, str(e), m, cid))
long = pd.DataFrame(rows, columns=["ANIO","REGION","EMPRESA","MONTO_CONTRATO","CONTRATO_ID"])

# Agregado por año-empresa: monto total y # de contratos distintos
edges = (long.groupby(["ANIO","REGION","EMPRESA"], as_index=False)
              .agg(monto_total=("MONTO_CONTRATO","sum"),
                   n_contratos=("CONTRATO_ID","nunique")))
YEARS = sorted(edges["ANIO"].unique())
print("Años encontrados:", YEARS)

Años encontrados: [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]


In [6]:
# ============ Métricas por año (HHI por contratos, #empresas, #contratos) ============
def stats_por_anio(y, region=REGION_OBJ):
    dfa = edges[(edges["ANIO"]==y) & (edges["REGION"]==region)].copy()
    if dfa.empty:
        return {"hhi_contratos": float("nan"), "n_empresas": 0, "n_contratos": 0}
    total_contr = dfa["n_contratos"].sum()
    shares = dfa["n_contratos"] / total_contr if total_contr>0 else 0
    hhi_contr = float(np.sum(np.square(shares))) if total_contr>0 else float("nan")
    return {
        "hhi_contratos": hhi_contr,
        "n_empresas": int(dfa["EMPRESA"].nunique()),
        "n_contratos": int(total_contr),
    }

In [7]:
# ============ Construcción de datos para HTML ============
def _build_graph_for_year(y, region=REGION_OBJ):
    dfa = edges[(edges["ANIO"]==y) & (edges["REGION"]==region)].copy()
    if dfa.empty:
        return [], [], {"hhi_contratos": float("nan"), "n_empresas": 0, "n_contratos": 0}

    st = stats_por_anio(y, region)

    # Nodos
    nodes = [{
        "id": region,
        "label": "Ancash",
        "shape": "circle",
        "size": 40,
        "color": "#1f77b4",
        "title": f"Gobierno Regional de Ancash ({y})<br>"
                 f"HHI (contratos): {st['hhi_contratos']:.3f}<br>"
                 f"Empresas: {st['n_empresas']} | Contratos: {st['n_contratos']}"
    }]
    for emp in dfa["EMPRESA"].unique().tolist():
        ncontr_emp = int(dfa.loc[dfa["EMPRESA"]==emp, "n_contratos"].sum())
        size = 10 + 7*ncontr_emp
        nodes.append({
            "id": emp,
            "label": str(emp),
            "shape": "dot",
            "size": size,
            "color": "#6baed6",
            "title": f"Empresa: {emp}"
        })

    # Aristas
    edges_list = []
    for _, r in dfa.iterrows():
        w = int(r["n_contratos"])
        width = 1 + 2.0*w
        title = f"# contratos: {w} | Monto total: S/ {float(r['monto_total']):,.2f}"
        edges_list.append({
            "from": region,
            "to": r["EMPRESA"],
            "value": w,
            "width": width,
            "title": title
        })

    return nodes, edges_list, st


In [8]:
# ============ Fallback HTML (vis-network) ============
def _write_visnetwork_html(nodes, edges, outfile_path, title_text):
    options = {
        "nodes": {"borderWidth": 1, "shape": "dot"},
        "edges": {"smooth": {"type": "dynamic"}},
        "interaction": {"hover": True, "tooltipDelay": 100},
        "physics": {"stabilization": {"iterations": 200}}
    }
    html = f"""<!DOCTYPE html>
<html lang="es">
<head>
<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width, initial-scale=1"/>
<title>{title_text}</title>
<style>
body{{margin:0;background:#ffffff;font-family:system-ui,-apple-system,Segoe UI,Roboto,Arial,sans-serif}}
#title{{padding:10px 12px;border-bottom:1px solid #eee}}
#network{{width:100vw;height:calc(100vh - 48px);}}
</style>
<link rel="preconnect" href="https://unpkg.com">
<script src="https://unpkg.com/vis-network/standalone/umd/vis-network.min.js"></script>
</head>
<body>
<div id="title">{title_text}</div>
<div id="network"></div>
<script>
const nodes = new vis.DataSet({json.dumps(nodes, ensure_ascii=False)});
const edges = new vis.DataSet({json.dumps(edges, ensure_ascii=False)});
const container = document.getElementById('network');
const data = {{ nodes, edges }};
const options = {json.dumps(options, ensure_ascii=False)};
const network = new vis.Network(container, data, options);
</script>
</body>
</html>"""
    Path(outfile_path).write_text(html, encoding="utf-8")

In [9]:
# ============ Guardado por año: PyVis con fallback ============
def save_year_html(y, region=REGION_OBJ, folder=SALIDA):
    dfa = edges[(edges["ANIO"]==y) & (edges["REGION"]==region)].copy()
    if dfa.empty:
        return None

    nodes, edges_list, st = _build_graph_for_year(y, region)
    title_text = f"Red de contratación — Ancash {y} (HHI por contratos)"

    folder.mkdir(parents=True, exist_ok=True)
    html_name = f"ancash_{y}.html"
    out_path = str(folder / html_name)

    # Intento PyVis
    try:
        from pyvis.network import Network
        net = Network(height="720px", width="100%", bgcolor="#ffffff", font_color="#222",
                      directed=False, notebook=False)
        # Física
        net.barnes_hut(gravity=-3500, central_gravity=0.12, spring_length=170,
                       spring_strength=0.01, damping=0.9)
        # Nodos
        for n in nodes:
            net.add_node(n["id"], label=n["label"], shape=n["shape"],
                         size=n["size"], color=n["color"], title=n["title"])
        # Aristas
        for e in edges_list:
            net.add_edge(e["from"], e["to"], value=e["value"],
                         width=e["width"], title=e["title"])
        # Opciones JSON válidas
        net.set_options("""
{
  "nodes": { "borderWidth": 1, "shape": "dot" },
  "edges": { "smooth": { "type": "dynamic" } },
  "interaction": { "hover": true, "tooltipDelay": 100 },
  "physics": { "stabilization": { "iterations": 200 } }
}
""")
        # Escribe sin abrir navegador
        net.write_html(out_path, notebook=False, open_browser=False)
        return html_name, st

    except Exception:
        # Fallback
        _write_visnetwork_html(nodes, edges_list, out_path, title_text)
        return html_name, st

In [10]:
# ============ Generar todos los años + índice (con casting a tipos nativos) ============
items_np = []
for y in YEARS:
    res = save_year_html(int(y))
    if res:
        fname, st = res
        # Convierte explícitamente a tipos nativos (int/float/None)
        hhi_val = None
        if pd.notna(st["hhi_contratos"]):
            try:
                hhi_val = float(st["hhi_contratos"])
            except Exception:
                hhi_val = None
        items_np.append({
            "y": int(y),
            "f": str(fname),
            "hhi": hhi_val,
            "emp": int(st["n_empresas"]),
            "contr": int(st["n_contratos"])
        })

print(f"Generados {len(items_np)} HTMLs en: {SALIDA.resolve()}")

# Index con selector y KPIs
SALIDA.mkdir(parents=True, exist_ok=True)
index_html = SALIDA / "index.html"

import json
items_js = json.dumps(items_np, ensure_ascii=False)  # ← ahora sí serializa

index_code = f"""<!DOCTYPE html>
<html lang="es">
<head>
<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width, initial-scale=1"/>
<title>Redes de contratación - Ancash (HHI por contratos)</title>
<style>
body{{font-family:system-ui,-apple-system,Segoe UI,Roboto,Arial,sans-serif;margin:0;padding:18px;background:#fafafa}}
.container{{max-width:1200px;margin:0 auto}}
h1{{font-size:20px;margin:0 0 14px}}
.bar{{display:flex;gap:10px;align-items:center;flex-wrap:wrap;margin-bottom:12px}}
label,select,button,small,span{{font-size:14px}}
iframe{{width:100%;height:78vh;border:1px solid #e3e3e3;border-radius:10px;background:#fff}}
button{{padding:6px 10px;border:1px solid #ddd;border-radius:8px;background:#fff;cursor:pointer}}
button:hover{{background:#f2f2f2}}
.kpi{{background:#fff;border:1px solid #e7e7e7;border-radius:8px;padding:6px 10px}}
.kpi b{{font-weight:600}}
small{{color:#666}}
</style>
</head>
<body>
<div class="container">
  <h1>Redes de contratación • Ancash (HHI por contratos)</h1>
  <div class="bar">
    <button onclick="prev()">⟨ Anterior</button>
    <label for="year">Año:</label>
    <select id="year" onchange="go(this.value)"></select>
    <button onclick="next()">Siguiente ⟩</button>
    <span class="kpi" id="kpi">—</span>
    <small>Grosor y tamaño = # de contratos; tooltip muestra #contratos y monto total.</small>
  </div>
  <iframe id="view" src="" loading="lazy"></iframe>
</div>
<script>
const items = {items_js};
let idx = 0;

function fillSelect() {{
  const sel = document.getElementById('year');
  sel.innerHTML = "";
  items.forEach((it, i) => {{
    const o = document.createElement('option');
    o.value = i;
    o.textContent = it.y;
    sel.appendChild(o);
  }});
}}

function updateKPI() {{
  const it = items[idx];
  const hhi = (it.hhi !== null && !isNaN(it.hhi)) ? Number(it.hhi).toFixed(3) : "—";
  document.getElementById('kpi').innerHTML = `<b>HHI (contratos):</b> ${hhi} | <b>Empresas:</b> ${it.emp} | <b>Contratos:</b> ${it.contr}`;
}}

function load() {{
  if (!items.length) return;
  const it = items[idx];
  document.getElementById('view').src = it.f;
  document.getElementById('year').selectedIndex = idx;
  updateKPI();
}}

function go(i) {{ idx = parseInt(i); load(); }}
function prev() {{ idx = (idx - 1 + items.length) % items.length; load(); }}
function next() {{ idx = (idx + 1) % items.length; load(); }}

fillSelect();
load();
</script>
</body>
</html>"""

index_html.write_text(index_code, encoding="utf-8")

print("✅ Index generado:")
print((SALIDA / "index.html").resolve())

Generados 20 HTMLs en: C:\Users\USUARIO\Documents\GitHub\Tesis_Redes\Tesis_1\docs


NameError: name 'hhi' is not defined

In [12]:
# --- Rehacer index.html sin f-string para evitar conflictos con ${...} de JavaScript
index_html = SALIDA / "index.html"

index_code = """<!DOCTYPE html>
<html lang="es">
<head>
<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width, initial-scale=1"/>
<title>Redes de contratación - Ancash (HHI por contratos)</title>
<style>
body{font-family:system-ui,-apple-system,Segoe UI,Roboto,Arial,sans-serif;margin:0;padding:18px;background:#fafafa}
.container{max-width:1200px;margin:0 auto}
h1{font-size:20px;margin:0 0 14px}
.bar{display:flex;gap:10px;align-items:center;flex-wrap:wrap;margin-bottom:12px}
label,select,button,small,span{font-size:14px}
iframe{width:100%;height:78vh;border:1px solid #e3e3e3;border-radius:10px;background:#fff}
button{padding:6px 10px;border:1px solid #ddd;border-radius:8px;background:#fff;cursor:pointer}
button:hover{background:#f2f2f2}
.kpi{background:#fff;border:1px solid #e7e7e7;border-radius:8px;padding:6px 10px}
.kpi b{font-weight:600}
small{color:#666}
</style>
</head>
<body>
<div class="container">
  <h1>Redes de contratación • Ancash (HHI por contratos)</h1>
  <div class="bar">
    <button onclick="prev()">⟨ Anterior</button>
    <label for="year">Año:</label>
    <select id="year" onchange="go(this.value)"></select>
    <button onclick="next()">Siguiente ⟩</button>
    <span class="kpi" id="kpi">—</span>
    <small>Grosor y tamaño = # de contratos; tooltip muestra #contratos y monto total.</small>
  </div>
  <iframe id="view" src="" loading="lazy"></iframe>
</div>
<script>
const items = __ITEMS__;
let idx = 0;

function fillSelect() {
  const sel = document.getElementById('year');
  sel.innerHTML = "";
  items.forEach((it, i) => {
    const o = document.createElement('option');
    o.value = i;
    o.textContent = it.y;
    sel.appendChild(o);
  });
}

function updateKPI() {
  const it = items[idx];
  const hhi = (it.hhi !== null && !isNaN(it.hhi)) ? Number(it.hhi).toFixed(3) : "—";
  document.getElementById('kpi').innerHTML = `<b>HHI (contratos):</b> ${hhi} | <b>Empresas:</b> ${it.emp} | <b>Contratos:</b> ${it.contr}`;
}

function load() {
  if (!items.length) return;
  const it = items[idx];
  document.getElementById('view').src = it.f;
  document.getElementById('year').selectedIndex = idx;
  updateKPI();
}

function go(i) { idx = parseInt(i); load(); }
function prev() { idx = (idx - 1 + items.length) % items.length; load(); }
function next() { idx = (idx + 1) % items.length; load(); }

fillSelect();
load();
</script>
</body>
</html>"""

# Inserta el JSON ya serializado sin romper los ${...} de JS
index_code = index_code.replace("__ITEMS__", items_js)

index_html.write_text(index_code, encoding="utf-8")
print("✅ Index regenerado:", index_html.resolve())

✅ Index regenerado: C:\Users\USUARIO\Documents\GitHub\Tesis_Redes\Tesis_1\docs\index.html
